In [1]:
import sqlite3

# Connect to the database
db_path = '/content/sample_data/Blooms.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Get list of tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [row[0] for row in cursor.fetchall()]

print("Table Summary (One row per table):\n")
print(f"{'Table Name':<12} | {'Row Count':<10} | Columns")
print("-" * 80)

# For each table, get column names and row count
for table in tables:
    # Get row count
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    row_count = cursor.fetchone()[0]

    # Get column names
    cursor.execute(f"PRAGMA table_info({table});")
    columns = [col[1] for col in cursor.fetchall()]
    column_str = ', '.join(columns)

    # Display in row
    print(f"{table:<12} | {row_count:<10} | {column_str}")

# Close connection
conn.close()


Table Summary (One row per table):

Table Name   | Row Count  | Columns
--------------------------------------------------------------------------------
FlanCSV      | 100        | resume_path, person_name, current_role, total_experience, current_tenure, certification_count, publication_count, skill_density
GPTCSV       | 100        | resume_path, person_name, current_role, total_experience, current_tenure, certification_count, publication_count, skill_density
LLamaCSV     | 100        | resume_path, person_name, current_role, total_experience, current_tenure, certification_count, publication_count, skill_density
NLPCSV       | 100        | resume_path, person_name, current_role, total_experience, current_tenure, certification_count, publication_count, skill_density
GTCSV        | 100        | resume_path, person_name, current_role, total_experience, current_tenure, certification_count, publication_count, skill_density


In [ ]:
import sqlite3
import pandas as pd

# Path to your database file
db_path = "/content/sample_data/Blooms.db"

# Connect and read the NLP table
conn = sqlite3.connect(db_path)
nlp_df = pd.read_sql_query("SELECT * FROM NLPCSV", conn)
conn.close()

# Display top rows
print("📄 NLP Resume Table from Database:")
nlp_df.head(10)


In [3]:
!pip install python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.1 MB/s eta 0:00:00


In [12]:
# step1_to_3_extract_jd_features.py

from docx import Document
import os
import re

def extract_text_from_docx(file_path):
    """Extracts all non-empty lines from a .docx file."""
    doc = Document(file_path)
    return "\n".join([p.text.strip() for p in doc.paragraphs if p.text.strip()])

def extract_jd_features(jd_text):
    """Extracts structured features from raw JD text."""
    jd_text_lower = jd_text.lower()
    lines = jd_text.strip().splitlines()

    # Extract target role (first non-empty line)
    target_role = next((line.strip() for line in lines if line.strip()), "Unknown")

    # Extract minimum experience (fallback = 1)
    exp_match = re.search(r'(\d+)\+?\s+years? of experience', jd_text_lower)
    min_experience = int(exp_match.group(1)) if exp_match else 1

    # Define skill keywords
    skill_keywords = [
        "python", "sql", "excel", "communication", "leadership",
        "project management", "data analysis", "crm", "cold calling"
    ]

    expected_skills = [skill for skill in skill_keywords if skill in jd_text_lower]
    skill_density = len(expected_skills)

    # Binary flags
    cert_required = "certification" in jd_text_lower
    pub_required = "publication" in jd_text_lower or "research paper" in jd_text_lower

    return {
        "expected_skills": expected_skills,
        "cert_required": cert_required,
        "pub_required": pub_required,
        "min_experience": min_experience,
        "target_role": target_role
    }

# === RUN THIS MODULE INTERACTIVELY ===

if __name__ == "__main__":
    jd_file_path = input("📂 Enter full path to your JD (.docx) file: ").strip()

    if not os.path.exists(jd_file_path):
        print("❌ File does not exist. Exiting.")
        exit()

    jd_text = extract_text_from_docx(jd_file_path)
    print("\n📄 Extracted JD Text:\n")
    print(jd_text[:1000] + "\n..." if len(jd_text) > 1000 else jd_text)

    jd_features = extract_jd_features(jd_text)

    print("\n✅ Extracted JD Features for AHP:")
    for k, v in jd_features.items():
        print(f"{k:<18}: {v}")


📂 Enter full path to your JD (.docx) file: /content/sample_data/Business Development Executive.docx

📄 Extracted JD Text:

Business Development Executive
About the Position
We are seeking proactive and driven Business Development Executives to join our admissions team. This role is instrumental in promoting our specialized Postgraduate, Master’s, and Executive Certification Programs designed for working professionals across industries. You will work alongside the Business Development Manager to generate leads, engage with prospective learners, and drive admissions conversions.
Job Role and Responsibilities
Proactively identify and engage potential candidates via inbound/outbound calls, cold calling, and email campaigns
Qualify leads based on eligibility and interest, and collaborate with the Business Development Manager for timely follow-ups and conversions
Recommend suitable programs to prospective learners based on their professional background and career goals
Maintain detailed and 

In [13]:
# step4_compute_fitment.py
import sqlite3
import pandas as pd
import numpy as np
import os
from difflib import SequenceMatcher

# === AHP Setup ===
pairwise_matrix = np.array([
    [1,     4,     3,     5,     3,     4],     # experience
    [1/4,   1,     1/2,   2,     1/2,   1],     # certification
    [1/3,   2,     1,     3,     1,     2],     # tenure
    [1/5,   1/2,   1/3,   1,     1/3,   1],     # publication
    [1/3,   2,     1,     3,     1,     2],     # skill_density
    [1/4,   1,     1/2,   1,     1/2,   1]      # role_match
])

criteria = ['experience', 'certification', 'tenure', 'publication', 'skill_density', 'role_match']

def compute_ahp_weights(matrix, labels):
    eigvals, eigvecs = np.linalg.eig(matrix)
    max_index = np.argmax(eigvals.real)
    principal_eigvec = eigvecs[:, max_index].real
    normalized_weights = principal_eigvec / principal_eigvec.sum()
    return dict(zip(labels, np.round(normalized_weights, 4)))

weights = compute_ahp_weights(pairwise_matrix, criteria)

# === JD Features (Set this for your JD) ===
jd_features = {
    "expected_skills": ["python", "machine learning", "sql"],
    "cert_required": True,
    "pub_required": False,
    "min_experience": 2,
    "target_role": "data analyst"
}

def compute_ahp_fitment_score(resume_features, jd_features, weights):
    resume_skills = set([s.lower() for s in resume_features.get('skills', [])]) if resume_features.get('skills') else set()
    expected_skills = set([s.lower() for s in jd_features.get('expected_skills', [])])
    S = len(resume_skills & expected_skills) / len(expected_skills) if expected_skills else 0
    C = 1 if jd_features['cert_required'] and resume_features.get('certification_count', 0) > 0 else 0
    P = 1 if jd_features['pub_required'] and resume_features.get('publication_count', 0) > 0 else 0
    E = min(resume_features.get('total_experience', 0) / jd_features['min_experience'], 1) if jd_features['min_experience'] > 0 else 1
    T = min(resume_features.get('current_tenure', 0) / 1, 1)
    R = SequenceMatcher(None, resume_features.get('current_role', '').lower(), jd_features.get('target_role', '').lower()).ratio()
    score = (
        weights['experience'] * E +
        weights['certification'] * C +
        weights['tenure'] * T +
        weights['publication'] * P +
        weights['skill_density'] * S +
        weights['role_match'] * R
    )
    return int(round(score * 100))

def classify_fit(score):
    if score < 31:
        return "Low Fit"
    elif score < 71:
        return "Medium Fit"
    else:
        return "Fit"

# === Fitment CSV Generation ===
DB_PATH = r"/content/sample_data/Blooms.db"
OUTPUT_DIR = "/content/sample_data/fitment_reports"
os.makedirs(OUTPUT_DIR, exist_ok=True)

model_tables = ["FlanCSV", "GPTCSV", "LLamaCSV", "NLPCSV", "GTCSV"]

def parse_skills_field(field):
    try:
        # Assuming a comma-separated skill string or list-like string
        if isinstance(field, str):
            return [s.strip() for s in field.split(",")]
        elif isinstance(field, list):
            return field
    except:
        return []
    return []

def process_table(table, conn, jd_features, weights):
    df = pd.read_sql_query(f"SELECT * FROM {table}", conn)

    required_cols = {
        "resume_path", "person_name", "total_experience", "current_tenure",
        "certification_count", "publication_count", "skill_density", "current_role"
    }

    if not required_cols.issubset(df.columns):
        print(f"⚠️ Skipping {table} — missing required columns.")
        return

    # Parse skills if available
    if "skills" in df.columns:
        df["skills_list"] = df["skills"].apply(parse_skills_field)
    else:
        df["skills_list"] = [[] for _ in range(len(df))]

    df["fitment_score"] = df.apply(
        lambda row: compute_ahp_fitment_score(row.to_dict(), jd_features, weights), axis=1
    )
    df["fitment_level"] = df["fitment_score"].apply(classify_fit)

    out_df = df[["resume_path", "person_name", "fitment_level", "fitment_score"]]
    out_file = os.path.join(OUTPUT_DIR, f"{table}_AHP_fitment.csv")
    out_df.to_csv(out_file, index=False)
    print(f"✅ Saved: {out_file}")

# === Run for all model tables ===
conn = sqlite3.connect(DB_PATH)
print("\n📊 Computing AHP-based fitment for all models...\n")
for table in model_tables:
    process_table(table, conn, jd_features, weights)
conn.close()



📊 Computing AHP-based fitment for all models...

✅ Saved: /content/sample_data/fitment_reports/FlanCSV_AHP_fitment.csv
✅ Saved: /content/sample_data/fitment_reports/GPTCSV_AHP_fitment.csv
✅ Saved: /content/sample_data/fitment_reports/LLamaCSV_AHP_fitment.csv
✅ Saved: /content/sample_data/fitment_reports/NLPCSV_AHP_fitment.csv
✅ Saved: /content/sample_data/fitment_reports/GTCSV_AHP_fitment.csv
